# Anime image generation

In [ ]:
import tensorflow as tf
import numpy as np
import sys
import os
import cv2
import glob
from PIL import Image
import matplotlib.pyplot as plt
import time
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import UpSampling2D, Conv2D

# Loading data

In [ ]:
! wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1z7rXRIFtRBFZHt-Mmti4HxrxHqUfG3Y8' -O tf-book.zip

In [ ]:
!unzip tf-book.zip

# Creating dataset

In [ ]:
def load_dataset(batch_size, img_shape, data_dir=None):
    # Create a tuple of size(30000,64,64,3)
    sample_dim = (batch_size,) + img_shape   
    # Create an uninitialized array of shape (30000,64,64,3)  
    sample = np.empty(sample_dim, dtype=np.float32) 
    # Extract all images from our file
    all_data_dirlist = list(glob.glob(data_dir)) 
    
    # Randomly select an image file from our data list
    sample_imgs_paths = np.random.choice(all_data_dirlist,batch_size) 
  
    for index,img_filename in enumerate(sample_imgs_paths):
        # Open the image
        image = Image.open(img_filename) 
        # Resize the image
        image = image.resize(img_shape[:-1]) 
        # Convert the input into an array
        image = np.asarray(image) 
        # Normalize data
        image = (image/127.5) -1 
        # Assign the preprocessed image to our sample
        sample[index,...] = image  
    print('data loaded')
    return sample


In [ ]:
x_train=load_dataset(30000,(64,64,3), "/content/tf-book/chapter13/anime/data/*.png")
BUFFER_SIZE = 30000
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Displaying data

In [ ]:
n = 10
f = plt.figure(figsize=(15,15))
for i in range(n):
    f.add_subplot(1, n, i + 1)
    plt.subplot(1, n, i+1 ).axis("off")
    plt.imshow(x_train[i])
plt.show()

In [ ]:
x_train.shape

# Creating generator model

In [ ]:
gen_model = tf.keras.Sequential()

# seed image of size 4x4
gen_model.add(tf.keras.layers.Dense(64*4*4, 
                                use_bias=False, 
                                input_shape=(100,)))
gen_model.add(tf.keras.layers.BatchNormalization())
gen_model.add(tf.keras.layers.LeakyReLU())
      
gen_model.add(tf.keras.layers.Reshape((4,4,64)))

# size of output image is still 4x4
gen_model.add(tf.keras.layers.Conv2DTranspose(256, (5, 5), 
                                          strides=(1, 1), 
                                          padding='same', 
                                          use_bias=False))
gen_model.add(tf.keras.layers.BatchNormalization())
gen_model.add(tf.keras.layers.LeakyReLU())

# size of output image is 8x8
gen_model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), 
                                          strides=(2, 2), 
                                          padding='same', 
                                          use_bias=False))
gen_model.add(tf.keras.layers.BatchNormalization())
gen_model.add(tf.keras.layers.LeakyReLU())

# size of output image is 16x16
gen_model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), 
                                          strides=(2, 2), 
                                          padding='same', 
                                          use_bias=False))
gen_model.add(tf.keras.layers.BatchNormalization())
gen_model.add(tf.keras.layers.LeakyReLU())

# size of output image is 32x32
gen_model.add(tf.keras.layers.Conv2DTranspose(32, (5, 5), 
                                          strides=(2, 2), 
                                          padding='same', 
                                          use_bias=False))
gen_model.add(tf.keras.layers.BatchNormalization())
gen_model.add(tf.keras.layers.LeakyReLU())

# size of output image is 64x64
gen_model.add(tf.keras.layers.Conv2DTranspose(3, (5, 5), 
                                          strides=(2, 2), 
                                          padding='same', 
                                          use_bias=False, 
                                          activation='tanh'))

gen_model.summary()


# Testing image generator with random input vector

In [ ]:
noise = tf.random.normal([1, 100])
generated_image = gen_model(noise, training=False)
plt.imshow(generated_image[0, :, :, 0] )

# Defining descriminator model

In [ ]:
discri_model = tf.keras.Sequential()
discri_model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same',input_shape=[64,64,3]))
discri_model.add(tf.keras.layers.LeakyReLU())
discri_model.add(tf.keras.layers.Dropout(0.3))
  
discri_model.add(tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
discri_model.add(tf.keras.layers.LeakyReLU())
discri_model.add(tf.keras.layers.Dropout(0.3))
    
discri_model.add(tf.keras.layers.Flatten())
discri_model.add(tf.keras.layers.Dense(1))
discri_model.summary()

In [ ]:
tf.keras.utils.plot_model(discri_model)

# Testing discriminator

In [ ]:
#giving the generated image to discriminator, the discriminator will give negative value if it is fake, while if it is real then it will give positive value.
decision = discri_model(generated_image)
print (decision)

# Loss functions

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(generated_output):
    return cross_entropy(tf.ones_like(generated_output),generated_output)

In [ ]:
def discriminator_loss(real_output, generated_output):
    # compute loss considering the image is real [1,1,...,1]
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)

    # compute loss considering the image is fake[0,0,...,0]
    generated_loss = cross_entropy(tf.zeros_like(generated_output),
                                   generated_output)
    # compute total loss
    total_loss = real_loss + generated_loss

    return total_loss

# Optimizers

In [ ]:
gen_optimizer = tf.optimizers.Adam(1e-4)
discri_optimizer = tf.optimizers.Adam(1e-4)

## Setting up a few variables

In [ ]:
epoch_number = 0
EPOCHS = 10000
noise_dim = 100
seed = tf.random.normal([1, noise_dim])

In [ ]:
checkpoint_dir = '/content/drive/My Drive/GAN3/Checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_optimizer,
                                 discriminator_optimizer=discri_optimizer, 
                                 generator= gen_model,
                                 discriminator = discri_model)

# Mounting drive for storing images and checkpoints

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/My Drive/GAN3'

# Gradient tuning function

In [ ]:
def gradient_tuning(images):
    # create a noise vector.
    noise = tf.random.normal([16, noise_dim])

    # Use gradient tapes for automatic differentiation 
    with tf.GradientTape() as generator_tape, tf.GradientTape() as discriminator_tape: 

      # ask genertor to generate random images
      generated_images = gen_model(noise, training=True)

      # ask discriminator to evalute the real images and generate its output
      real_output = discri_model(images, training=True)

      # ask discriminator to do the evlaution on generated (fake) images
      fake_output = discri_model(generated_images, training=True)

      # calculate generator loss on fake data
      gen_loss = generator_loss(fake_output)

      # calculate discriminator loss as defined earlier
      disc_loss = discriminator_loss(real_output, fake_output)

    # calculate gradients for generator
    gen_gradients = generator_tape.gradient(gen_loss, 
                                               gen_model.trainable_variables)

    # calculate gradients for discriminator
    discri_gradients = discriminator_tape.gradient(disc_loss, 
                                        discri_model.trainable_variables)

    # use optimizer to process and apply gradients to variables
    gen_optimizer.apply_gradients(zip(gen_gradients, 
                                            gen_model.trainable_variables))
    
    # same as above to discriminator
    discri_optimizer.apply_gradients(
        zip(discri_gradients, 
            discri_model.trainable_variables))

# Function for generating images at every epoch

In [ ]:
    def generate_and_save_images(model, epoch, test_input):
        # use a global count for tracking epochs in case of disconnection
        global epoch_number
        epoch_number = epoch_number + 1

        # set training to false to ensure inference mode
        predictions = model(test_input, training=False)

        # display and save image
        fig = plt.figure(figsize=(4,4))
        for i in range(predictions.shape[0]):
            plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.axis('off')
        plt.savefig('image_at_epoch_{:01d}.png'.format(epoch_number))
        plt.show()

# Setting up a training loop

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      gradient_tuning(image_batch)
    
    # Produce images as we go
    generate_and_save_images(gen_model,
                             epoch + 1,
                             seed)
    
    # save checkpoint data
    checkpoint.save(file_prefix = checkpoint_prefix)
    print ('Time for epoch {} is {} sec'.format(epoch + 1, 
                                                time.time()-start))

# Model training

In [ ]:
train(train_dataset, EPOCHS)

### Run following code only if there is a disconnection and you wish to continue training from the last checkpoint

In [ ]:
#run this code only if there is a runtime disconnection
try:
     checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
except Exception as error:
    print("Error loading in model : {}".format(error))
train(train_dataset, EPOCHS)